In [1]:
!git clone --recursive https://github.com/Microsoft/LightGBM
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

Cloning into 'LightGBM'...
remote: Enumerating objects: 128, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 20659 (delta 66), reused 49 (delta 21), pack-reused 20531
Receiving objects: 100% (20659/20659), 16.43 MiB | 29.31 MiB/s, done.
Resolving deltas: 100% (15082/15082), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728        
Receiving objects: 100% (21728/2

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import DataFrame, concat
!pip install import_ipynb
import import_ipynb
from google.colab import drive
drive.mount('/content/drive')
from datetime import date
import lightgbm as lgbm
from lightgbm import LGBMRegressor
import time
import statsmodels.api as sm
from sklearn import linear_model, metrics
from sklearn.model_selection import train_test_split
from tqdm import tnrange, tqdm_notebook
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import os
from sklearn.decomposition import PCA

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=62fa032a256d32890e07c728f1e5e93c3c758f36f3e6f86e3958d2b2abd1bf94
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
Mounted at /content/drive


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
import math
from math import radians
from sklearn.preprocessing import StandardScaler

In [4]:
#### 파라미터 ####

# 하루의 틱
ticks = 48

# 예측에 사용할 일수
days = 3                # 이 부분을 바꿀 수 있다.
n_days = ticks*days

# 미래 예측할 일수
future_days = 2
future_window = ticks * future_days

### 모든변수
# ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']

# 사용할 변수
what_to_left = ['Time', 'WS', 'DHI','DNI','RH','T','TARGET', 'Day']

n_features = len(what_to_left)
n_obs = n_days * n_features # 7일 예측 기준으로 7(일 수) * 48(틱 수) * 6(변수 개수)

In [5]:
# 데이터 불러오기 

raw_data = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/train/train.csv')
submission = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/sample_submission.csv')
submission.shape

(7776, 10)

In [6]:
def drop_clms(dataset):
  dataset['Time'] = dataset['Hour'] + dataset['Minute']*(0.5/30)
  dataset['Date'] = dataset['Day']%365
  dataset.drop('Day', axis=1, inplace=True)
  
  temp = list()
  for i in range(0, len(dataset), 48):
    temp += [24-(list(dataset.DHI[i:i+48]).count(0)*0.5)]*48 

  dataset["SH"] = temp

  return dataset

def cos_time(dataset):
  dataset['sin_time'] = np.sin(2*np.pi*dataset.Time/24)
  dataset['cos_time'] = np.cos(2*np.pi*dataset.Time/24)
  return dataset

In [7]:
def get_yday(when):
  HP = date(2020,1,1)
  results = (when-HP).days
  return results

In [8]:
def 절기24(Date):
  target = 0
  for i in list24:
    if Date < i:
      target = list24.index(i) - 1
      break
  if Date < 5:
    target = 23
  return target

In [9]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def train_to_supervised(train, target, n_in):

    clmns = list(train.columns)

    # 기타 칼럼은 전과 같이 들어갈 것.
    scaled_lst = clmns

    scaled_df = train[scaled_lst]
    target_df = target

    # 미래 몇 번째 항목을 가져올 것인가
    future = [48, 96]

    ### 만약에 스케일링을 하고 싶다면 ###
    # scaled_df 데이터 프레임만 스케일링 하고, 절기랑 TARGET 데이터는 그냥 두면 된다.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    n_vars2 = 1 if type(target_df) is list else target_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]

    # 48과 96 후의 타겟 데이터 2개 붙이기.
    # forecast sequence (t, t+1, ... t+n)
    for i in future:
        cols.append(target_df.shift(-i))
        if i == 0:
            names += [('TARGET%d(t)' % (j+1)) for j in range(n_vars2)]
        else:
            names += [('TARGET%d(t+%d)' % (j+1, i)) for j in range(n_vars2)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [10]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def test_to_supervised(train, n_in):

    clmns = list(train.columns)

    # 기타 칼럼은 전과 같이 들어갈 것.
    scaled_lst = clmns
    scaled_df = train[scaled_lst]

    ### 만약에 스케일링을 하고 싶다면 ###
    # testset의 스케일링은 구조가 상당히 까다로우므로....... 일단 나중에 하기로 함.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]
    
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [11]:
소한 = get_yday(date(2020,1,6))
대한 = get_yday(date(2020,1,20))
입춘 = get_yday(date(2020,2,4))
우수 = get_yday(date(2020,2,19))
경칩 = get_yday(date(2020,3,6))
춘분 = get_yday(date(2020,3,21))
청명 = get_yday(date(2020,4,5))
곡우 = get_yday(date(2020,4,20))
입하 = get_yday(date(2020,5,6))
소만 = get_yday(date(2020,5,21))
망종 = get_yday(date(2020,6,6))
하지 = get_yday(date(2020,6,22))
소서 = get_yday(date(2020,7,7))
대서 = get_yday(date(2020,7,23))
입추 = get_yday(date(2020,8,8))
처서 = get_yday(date(2020,8,23))
백로 = get_yday(date(2020,9,8))
추분 = get_yday(date(2020,9,23))
한로 = get_yday(date(2020,10,8))
상강 = get_yday(date(2020,10,24))
입동 = get_yday(date(2020,11,8))
소설 = get_yday(date(2020,11,22))
대설 = get_yday(date(2020,12,7))
동지 = get_yday(date(2020,12,22))

list24 = [소한, 대한, 입춘, 우수, 경칩, 춘분, 청명, 곡우, 입하, 소만, 망종, 하지, 소서, 대서, 입추, 처서, 백로, 추분, 한로, 상강, 입동, 소설, 대설, 동지]

In [12]:
m = 60

일출 = [7+47/m, 7+44/m, 7+34/m, 7+18/m, 6+56/m, 6+35/m, 6+12/m, 5+51/m, 5+33/m, 5+19/m, 5+11/m, 5+11/m, 5+17/m, 5+27/m, 5+41/m, 5+54/m, 6+8/m, 6+20/m, 6+33/m, 6+47/m, 7+3/m, 7+20/m, 7+33/m, 7+45/m]
일몰 = [17+28/m, 17+42/m, 17+58/m, 18+15/m, 18+30/m, 18+44/m, 18+58/m, 19+12/m, 19+25/m, 19+39/m, 19+50/m, 19+57/m, 19+56/m, 19+50/m, 19+35/m, 19+15/m, 18+51/m, 18+28/m, 18+6/m, 17+45/m, 17+28/m, 17+17/m, 17+13/m, 17+17/m]
경사각 = [32.92, 36.83, 40.75, 44.67, 48.58, 52.5, 56.42, 60.33, 64.25, 68.16, 72.01, 76, 72.1, 68.16, 64.25, 60.33, 56.42, 52.5, 48.58, 44.67, 40.75, 36.83, 32.92, 29]
남중 = [12+36/m, 12+41/m, 12+44/m, 12+44/m, 12+42/m, 12+38/m, 12+34/m, 12+29/m, 12+27/m, 12+27/m, 12+29/m, 12+32/m, 12+35/m, 12+37/m, 12+36/m, 12+33/m, 12+29/m, 12+23/m, 12+18/m, 12+15/m, 12+14/m, 12+16/m, 12+22/m, 12+29/m]
print(len(일출), len(일몰), len(경사각), len(남중))

24 24 24 24


In [13]:
def HRA(DHI, DNI, season, hour):
  # 위도(latitude) 기준을 일단 임의로 대전으로 설정 (위도 36.19~36.2도)
  latitude = radians(36.2)
  season = int(season)
  # 절기별 대한민국의 경사각
  tilt = radians(경사각[season])

  # 절기별 대한민국 대전의 태양 남중시각
  hra = radians(15*(hour - 남중[season]))

  # 구하려는 알파
  elevation = np.arcsin(np.sin(tilt) * np.sin(latitude) + np.cos(tilt) * np.cos(latitude) * np.cos(hra))

  # 천정각(Zenith Angle)은 90 - 알파
  zenith = radians(90) - elevation

  # GHI는 DHI + DNI * cos(천정각)
  ghi = DHI + DNI *np.cos(zenith)

  return ghi

In [14]:
def not_minus(dataset):
  for i in range(0, len(dataset.index)):
    for j in range(0, len(dataset.columns)):
      K = dataset.iloc[i,j]
      if K < 0:
        dataset.iloc[i,j] = 0
  return dataset

In [15]:
dataset = raw_data.copy()
dataset2 = drop_clms(dataset)
dataset2.drop(['Hour', 'Minute'], axis =1, inplace=True)
dataset2

,DHI,DNI,WS,RH,T,TARGET,Time,Date,SH
0,0,0,1.5,69.08,-12,0.0,0.0,0,9.0
1,0,0,1.5,69.06,-12,0.0,0.5,0,9.0
2,0,0,1.6,71.78,-12,0.0,1.0,0,9.0
3,0,0,1.6,71.75,-12,0.0,1.5,0,9.0
4,0,0,1.6,75.20,-12,0.0,2.0,0,9.0
...,...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,21.5,364,9.0
52556,0,0,2.4,66.79,-4,0.0,22.0,364,9.0
52557,0,0,2.2,66.78,-4,0.0,22.5,364,9.0
52558,0,0,2.1,67.72,-4,0.0,23.0,364,9.0


In [16]:
dataset2['season'] = dataset2.apply(lambda x: 절기24(x['Date']), axis = 1)
dataset3 = cos_time(dataset2)
dataset4 = dataset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]
# dataset5 = dataset4.iloc[n_days:,:-1]
# goals = pd.DataFrame(dataset4.Goal, index = dataset4.index)
dataset4

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time
0,0,0,1.5,69.08,-12,0.0,9.0,0.000000,1.000000,23,0.0
1,0,0,1.5,69.06,-12,0.0,9.0,0.130526,0.991445,23,0.5
2,0,0,1.6,71.78,-12,0.0,9.0,0.258819,0.965926,23,1.0
3,0,0,1.6,71.75,-12,0.0,9.0,0.382683,0.923880,23,1.5
4,0,0,1.6,75.20,-12,0.0,9.0,0.500000,0.866025,23,2.0
...,...,...,...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,9.0,-0.608761,0.793353,0,21.5
52556,0,0,2.4,66.79,-4,0.0,9.0,-0.500000,0.866025,0,22.0
52557,0,0,2.2,66.78,-4,0.0,9.0,-0.382683,0.923880,0,22.5
52558,0,0,2.1,67.72,-4,0.0,9.0,-0.258819,0.965926,0,23.0


In [17]:
dataset5= dataset4.copy()

dataset5['GHI'] = dataset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)

In [18]:
dataset5.iloc[50:51,:]

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time,GHI
50,0,0,1.6,88.11,-11,0.0,9.0,0.258819,0.965926,23,1.0,0.0


In [19]:
dataset6 = dataset5[[ 'DHI', 'DNI', 'WS', 'RH', 'T','SH', 'sin_time', 'cos_time', 'GHI', 'TARGET']] 
dataset6.head(10)

,DHI,DNI,WS,RH,T,SH,sin_time,cos_time,GHI,TARGET
0,0,0,1.5,69.08,-12,9.0,0.000000,1.000000,0.0,0.0
1,0,0,1.5,69.06,-12,9.0,0.130526,0.991445,0.0,0.0
2,0,0,1.6,71.78,-12,9.0,0.258819,0.965926,0.0,0.0
3,0,0,1.6,71.75,-12,9.0,0.382683,0.923880,0.0,0.0
4,0,0,1.6,75.20,-12,9.0,0.500000,0.866025,0.0,0.0
5,0,0,1.5,69.29,-11,9.0,0.608761,0.793353,0.0,0.0
6,0,0,1.5,72.56,-11,9.0,0.707107,0.707107,0.0,0.0
7,0,0,1.4,72.55,-11,9.0,0.793353,0.608761,0.0,0.0
8,0,0,1.3,74.62,-11,9.0,0.866025,0.500000,0.0,0.0
9,0,0,1.3,74.61,-11,9.0,0.923880,0.382683,0.0,0.0


In [20]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(dataset6)
x = pd.DataFrame(scaler.transform(dataset6))
x

,0,1,2,3,4,5,6,7,8,9
0,-0.619312,-0.671447,-0.670026,0.557162,-2.090439,-1.452834,1.300046e-17,1.414214,-0.74381,-0.690638
1,-0.619312,-0.671447,-0.670026,0.556255,-2.090439,-1.452834,1.845919e-01,1.402115,-0.74381,-0.690638
2,-0.619312,-0.671447,-0.599942,0.679596,-2.090439,-1.452834,3.660254e-01,1.366025,-0.74381,-0.690638
3,-0.619312,-0.671447,-0.599942,0.678235,-2.090439,-1.452834,5.411961e-01,1.306563,-0.74381,-0.690638
4,-0.619312,-0.671447,-0.599942,0.834679,-2.090439,-1.452834,7.071068e-01,1.224745,-0.74381,-0.690638
...,...,...,...,...,...,...,...,...,...,...
52555,-0.619312,-0.671447,-0.039270,0.630622,-1.304557,-1.452834,-8.609187e-01,1.121971,-0.74381,-0.690638
52556,-0.619312,-0.671447,-0.039270,0.453319,-1.304557,-1.452834,-7.071068e-01,1.224745,-0.74381,-0.690638
52557,-0.619312,-0.671447,-0.179438,0.452866,-1.304557,-1.452834,-5.411961e-01,1.306563,-0.74381,-0.690638
52558,-0.619312,-0.671447,-0.249522,0.495491,-1.304557,-1.452834,-3.660254e-01,1.366025,-0.74381,-0.690638


In [21]:
Pika = 0.97
pca = PCA(n_components= Pika, svd_solver='full' )
pca.fit(x)
temp = pd.DataFrame(pca.transform(x))

temp

,0,1,2,3,4,5,6
0,-2.671069,1.458776,-0.432031,-0.602431,0.455664,0.063409,-1.027970
1,-2.665764,1.497592,-0.264583,-0.631317,0.489789,0.015146,-1.011698
2,-2.685643,1.555243,-0.066205,-0.598667,0.511859,0.070568,-0.944999
3,-2.661912,1.599643,0.088703,-0.623662,0.558628,0.027086,-0.892075
4,-2.685768,1.637283,0.264445,-0.643180,0.623923,0.111919,-0.782004
...,...,...,...,...,...,...,...
52555,-2.265738,1.120312,-1.107117,0.091507,-0.009418,0.166003,-0.567995
52556,-2.242000,1.144347,-0.993231,0.058522,-0.029841,-0.020026,-0.687621
52557,-2.292761,1.119991,-0.857672,-0.087240,0.031478,-0.075105,-0.746093
52558,-2.340407,1.121027,-0.695708,-0.175007,0.075448,-0.093943,-0.775510


In [22]:
# 각각의 요인이 가지는 설명력

pca.explained_variance_ratio_

array([0.49271605, 0.15396043, 0.10360851, 0.08855306, 0.06984653,
       0.04646945, 0.02267433])

In [23]:
temp_Y = pd.DataFrame(dataset6.iloc[:,-1])

# scaler = MinMaxScaler(feature_range=(-1, 1))

# scaler.fit(dataset6)

# dataset7 = pd.DataFrame(scaler.transform(dataset6))

# dataset7.columns = dataset6.columns
# dataset7

## dataset6 가지고 동일하게 to_supervised 함수를 써서 나누면 됩니다.
### 나머지 변수 합치기, 인코딩은 유진이가 해줄 것...

In [24]:
dataset8 = train_to_supervised(temp, temp_Y,n_days)
dataset8

,0(t-144),1(t-144),2(t-144),3(t-144),4(t-144),5(t-144),6(t-144),0(t-143),1(t-143),2(t-143),3(t-143),4(t-143),5(t-143),6(t-143),0(t-142),1(t-142),2(t-142),3(t-142),4(t-142),5(t-142),6(t-142),0(t-141),1(t-141),2(t-141),3(t-141),4(t-141),5(t-141),6(t-141),0(t-140),1(t-140),2(t-140),3(t-140),4(t-140),5(t-140),6(t-140),0(t-139),1(t-139),2(t-139),3(t-139),4(t-139),...,4(t-6),5(t-6),6(t-6),0(t-5),1(t-5),2(t-5),3(t-5),4(t-5),5(t-5),6(t-5),0(t-4),1(t-4),2(t-4),3(t-4),4(t-4),5(t-4),6(t-4),0(t-3),1(t-3),2(t-3),3(t-3),4(t-3),5(t-3),6(t-3),0(t-2),1(t-2),2(t-2),3(t-2),4(t-2),5(t-2),6(t-2),0(t-1),1(t-1),2(t-1),3(t-1),4(t-1),5(t-1),6(t-1),TARGET1(t+48),TARGET1(t+96)
144,-2.671069,1.458776,-0.432031,-0.602431,0.455664,0.063409,-1.027970,-2.665764,1.497592,-0.264583,-0.631317,0.489789,0.015146,-1.011698,-2.685643,1.555243,-0.066205,-0.598667,0.511859,0.070568,-0.944999,-2.661912,1.599643,0.088703,-0.623662,0.558628,0.027086,-0.892075,-2.685768,1.637283,0.264445,-0.643180,0.623923,0.111919,-0.782004,-2.531228,1.624939,0.334884,-0.722196,0.667716,...,0.760073,0.809319,-0.585231,-2.890687,1.053695,-1.242648,-1.071936,0.775630,0.762353,-0.675929,-2.897706,1.128650,-1.130167,-1.107615,0.774805,0.608669,-0.824619,-2.899407,1.215944,-0.938260,-0.965577,0.689896,0.570456,-0.881755,-2.915273,1.326041,-0.710384,-0.764662,0.591737,0.606804,-0.897545,-2.899298,1.423620,-0.508010,-0.622791,0.523148,0.567120,-0.918248,0.0,0.0
145,-2.665764,1.497592,-0.264583,-0.631317,0.489789,0.015146,-1.011698,-2.685643,1.555243,-0.066205,-0.598667,0.511859,0.070568,-0.944999,-2.661912,1.599643,0.088703,-0.623662,0.558628,0.027086,-0.892075,-2.685768,1.637283,0.264445,-0.643180,0.623923,0.111919,-0.782004,-2.531228,1.624939,0.334884,-0.722196,0.667716,-0.165898,-0.733431,-2.536726,1.663965,0.481251,-0.735548,0.739954,...,0.775630,0.762353,-0.675929,-2.897706,1.128650,-1.130167,-1.107615,0.774805,0.608669,-0.824619,-2.899407,1.215944,-0.938260,-0.965577,0.689896,0.570456,-0.881755,-2.915273,1.326041,-0.710384,-0.764662,0.591737,0.606804,-0.897545,-2.899298,1.423620,-0.508010,-0.622791,0.523148,0.567120,-0.918248,-2.839773,1.607129,-0.325407,-0.428167,0.439544,0.441203,-0.988111,0.0,0.0
146,-2.685643,1.555243,-0.066205,-0.598667,0.511859,0.070568,-0.944999,-2.661912,1.599643,0.088703,-0.623662,0.558628,0.027086,-0.892075,-2.685768,1.637283,0.264445,-0.643180,0.623923,0.111919,-0.782004,-2.531228,1.624939,0.334884,-0.722196,0.667716,-0.165898,-0.733431,-2.536726,1.663965,0.481251,-0.735548,0.739954,-0.078232,-0.594975,-2.492489,1.687871,0.570011,-0.805565,0.831180,...,0.774805,0.608669,-0.824619,-2.899407,1.215944,-0.938260,-0.965577,0.689896,0.570456,-0.881755,-2.915273,1.326041,-0.710384,-0.764662,0.591737,0.606804,-0.897545,-2.899298,1.423620,-0.508010,-0.622791,0.523148,0.567120,-0.918248,-2.839773,1.607129,-0.325407,-0.428167,0.439544,0.441203,-0.988111,-2.786347,1.639345,-0.132764,-0.339122,0.396115,0.372355,-0.940632,0.0,0.0
147,-2.661912,1.599643,0.088703,-0.623662,0.558628,0.027086,-0.892075,-2.685768,1.637283,0.264445,-0.643180,0.623923,0.111919,-0.782004,-2.531228,1.624939,0.334884,-0.722196,0.667716,-0.165898,-0.733431,-2.536726,1.663965,0.481251,-0.735548,0.739954,-0.078232,-0.594975,-2.492489,1.687871,0.570011,-0.805565,0.831180,-0.105970,-0.481705,-2.476191,1.705224,0.658633,-0.870058,0.931629,...,0.689896,0.570456,-0.881755,-2.915273,1.326041,-0.710384,-0.764662,0.591737,0.606804,-0.897545,-2.899298,1.423620,-0.508010,-0.622791,0.523148,0.567120,-0.918248,-2.839773,1.607129,-0.325407,-0.428167,0.439544,0.441203,-0.988111,-2.786347,1.639345,-0.132764,-0.339122,0.396115,0.372355,-0.940632,-2.751195,1.726105,0.051854,-0.251220,0.378209,0.330871,-0.905831,0.0,0.0
148,-2.685768,1.637283,0.264445,-0.643180,0.623923,0.111919,-0.782004,-2.531228,1.624939,0.334884,-0.722196,0.667716,-0.165898,-0.733431,-2.536726,1.663965,0.481251,-0.735548,0.739954,-0.078232,-0.594975,-2.492489,1.687871,0.570011,-0.805565,0.831180,-0.105970,-0.481705,-2.476191,1.705224,0.658633,-0.8700

In [25]:
X = dataset8.iloc[:,:-2]

y_1 = pd.DataFrame(dataset8.iloc[:,-2])
y_2 = pd.DataFrame(dataset8.iloc[:,-1])
X

,0(t-144),1(t-144),2(t-144),3(t-144),4(t-144),5(t-144),6(t-144),0(t-143),1(t-143),2(t-143),3(t-143),4(t-143),5(t-143),6(t-143),0(t-142),1(t-142),2(t-142),3(t-142),4(t-142),5(t-142),6(t-142),0(t-141),1(t-141),2(t-141),3(t-141),4(t-141),5(t-141),6(t-141),0(t-140),1(t-140),2(t-140),3(t-140),4(t-140),5(t-140),6(t-140),0(t-139),1(t-139),2(t-139),3(t-139),4(t-139),...,2(t-6),3(t-6),4(t-6),5(t-6),6(t-6),0(t-5),1(t-5),2(t-5),3(t-5),4(t-5),5(t-5),6(t-5),0(t-4),1(t-4),2(t-4),3(t-4),4(t-4),5(t-4),6(t-4),0(t-3),1(t-3),2(t-3),3(t-3),4(t-3),5(t-3),6(t-3),0(t-2),1(t-2),2(t-2),3(t-2),4(t-2),5(t-2),6(t-2),0(t-1),1(t-1),2(t-1),3(t-1),4(t-1),5(t-1),6(t-1)
144,-2.671069,1.458776,-0.432031,-0.602431,0.455664,0.063409,-1.027970,-2.665764,1.497592,-0.264583,-0.631317,0.489789,0.015146,-1.011698,-2.685643,1.555243,-0.066205,-0.598667,0.511859,0.070568,-0.944999,-2.661912,1.599643,0.088703,-0.623662,0.558628,0.027086,-0.892075,-2.685768,1.637283,0.264445,-0.643180,0.623923,0.111919,-0.782004,-2.531228,1.624939,0.334884,-0.722196,0.667716,...,-1.368230,-0.985989,0.760073,0.809319,-0.585231,-2.890687,1.053695,-1.242648,-1.071936,0.775630,0.762353,-0.675929,-2.897706,1.128650,-1.130167,-1.107615,0.774805,0.608669,-0.824619,-2.899407,1.215944,-0.938260,-0.965577,0.689896,0.570456,-0.881755,-2.915273,1.326041,-0.710384,-0.764662,0.591737,0.606804,-0.897545,-2.899298,1.423620,-0.508010,-0.622791,0.523148,0.567120,-0.918248
145,-2.665764,1.497592,-0.264583,-0.631317,0.489789,0.015146,-1.011698,-2.685643,1.555243,-0.066205,-0.598667,0.511859,0.070568,-0.944999,-2.661912,1.599643,0.088703,-0.623662,0.558628,0.027086,-0.892075,-2.685768,1.637283,0.264445,-0.643180,0.623923,0.111919,-0.782004,-2.531228,1.624939,0.334884,-0.722196,0.667716,-0.165898,-0.733431,-2.536726,1.663965,0.481251,-0.735548,0.739954,...,-1.242648,-1.071936,0.775630,0.762353,-0.675929,-2.897706,1.128650,-1.130167,-1.107615,0.774805,0.608669,-0.824619,-2.899407,1.215944,-0.938260,-0.965577,0.689896,0.570456,-0.881755,-2.915273,1.326041,-0.710384,-0.764662,0.591737,0.606804,-0.897545,-2.899298,1.423620,-0.508010,-0.622791,0.523148,0.567120,-0.918248,-2.839773,1.607129,-0.325407,-0.428167,0.439544,0.441203,-0.988111
146,-2.685643,1.555243,-0.066205,-0.598667,0.511859,0.070568,-0.944999,-2.661912,1.599643,0.088703,-0.623662,0.558628,0.027086,-0.892075,-2.685768,1.637283,0.264445,-0.643180,0.623923,0.111919,-0.782004,-2.531228,1.624939,0.334884,-0.722196,0.667716,-0.165898,-0.733431,-2.536726,1.663965,0.481251,-0.735548,0.739954,-0.078232,-0.594975,-2.492489,1.687871,0.570011,-0.805565,0.831180,...,-1.130167,-1.107615,0.774805,0.608669,-0.824619,-2.899407,1.215944,-0.938260,-0.965577,0.689896,0.570456,-0.881755,-2.915273,1.326041,-0.710384,-0.764662,0.591737,0.606804,-0.897545,-2.899298,1.423620,-0.508010,-0.622791,0.523148,0.567120,-0.918248,-2.839773,1.607129,-0.325407,-0.428167,0.439544,0.441203,-0.988111,-2.786347,1.639345,-0.132764,-0.339122,0.396115,0.372355,-0.940632
147,-2.661912,1.599643,0.088703,-0.623662,0.558628,0.027086,-0.892075,-2.685768,1.637283,0.264445,-0.643180,0.623923,0.111919,-0.782004,-2.531228,1.624939,0.334884,-0.722196,0.667716,-0.165898,-0.733431,-2.536726,1.663965,0.481251,-0.735548,0.739954,-0.078232,-0.594975,-2.492489,1.687871,0.570011,-0.805565,0.831180,-0.105970,-0.481705,-2.476191,1.705224,0.658633,-0.870058,0.931629,...,-0.938260,-0.965577,0.689896,0.570456,-0.881755,-2.915273,1.326041,-0.710384,-0.764662,0.591737,0.606804,-0.897545,-2.899298,1.423620,-0.508010,-0.622791,0.523148,0.567120,-0.918248,-2.839773,1.607129,-0.325407,-0.428167,0.439544,0.441203,-0.988111,-2.786347,1.639345,-0.132764,-0.339122,0.396115,0.372355,-0.940632,-2.751195,1.726105,0.051854,-0.251220,0.378209,0.330871,-0.905831
148,-2.685768,1.637283,0.264445,-0.643180,0.623923,0.111919,-0.782004,-2.531228,1.624939,0.334884,-0.722196,0.667716,-0.165898,-0.733431,-2.536726,1.663965,0.481251,-0.735548,0.739954,-0.078232,-0.594975,-2.492489,1.687871,0.570011,-0.805565,0.831180,-0.105970,-0.481705,-

# testset은 이미 24절기(season)가 들어있으므로, 약간 다르게 전처리를 해 줘야 함.
## (season) 구하는 과정이 생략됨.
## 이하는 testset을 구하는 코드

In [26]:
file_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/' + str(1) + '.csv'
new_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/adj_test/' + str(1) + '.csv'
file_name = str(1) + '.csv'

# adj_test에서 가져와야 하므로 new_path에서 불러온다
temp = pd.read_csv(new_path)
temp

,Unnamed: 0,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,Time,24
0,0,0,0,0,0,0,2.8,70.34,-6.1,0.0,0.0,22
1,1,0,0,30,0,0,2.9,71.97,-6.4,0.0,0.5,22
2,2,0,1,0,0,0,2.9,69.77,-6.7,0.0,1.0,22
3,3,0,1,30,0,0,2.9,71.95,-7.1,0.0,1.5,22
4,4,0,2,0,0,0,2.9,69.48,-7.4,0.0,2.0,22
...,...,...,...,...,...,...,...,...,...,...,...,...
331,331,6,21,30,0,0,1.7,43.65,-5.2,0.0,21.5,22
332,332,6,22,0,0,0,1.5,40.62,-4.9,0.0,22.0,22
333,333,6,22,30,0,0,1.5,40.62,-4.9,0.0,22.5,22
334,334,6,23,0,0,0,1.5,39.21,-4.9,0.0,23.0,22


In [27]:
temp.drop('Unnamed: 0', axis = 1, inplace = True)

In [28]:
testset = temp.copy()

# 24로 저장한 24절기 season으로 이름 바꾸기
testset.rename(columns = {'24' : 'season'}, inplace = True)


# 이 다음은 trainset과 동일한 전처리, season은 이미 구해져 있으므로 구하지 않음
testset2 = drop_clms(testset)
testset2.drop(['Hour', 'Minute','Date'], axis=1,inplace=True)
testset3 = cos_time(testset2)
testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]
testset4

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time
0,0,0,2.8,70.34,-6.1,0.0,9.5,0.000000,1.000000,22,0.0
1,0,0,2.9,71.97,-6.4,0.0,9.5,0.130526,0.991445,22,0.5
2,0,0,2.9,69.77,-6.7,0.0,9.5,0.258819,0.965926,22,1.0
3,0,0,2.9,71.95,-7.1,0.0,9.5,0.382683,0.923880,22,1.5
4,0,0,2.9,69.48,-7.4,0.0,9.5,0.500000,0.866025,22,2.0
...,...,...,...,...,...,...,...,...,...,...,...
331,0,0,1.7,43.65,-5.2,0.0,9.5,-0.608761,0.793353,22,21.5
332,0,0,1.5,40.62,-4.9,0.0,9.5,-0.500000,0.866025,22,22.0
333,0,0,1.5,40.62,-4.9,0.0,9.5,-0.382683,0.923880,22,22.5
334,0,0,1.5,39.21,-4.9,0.0,9.5,-0.258819,0.965926,22,23.0


In [29]:
testset5 = testset4.copy()
testset5['GHI'] = testset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)

In [30]:
testset5.iloc[50:51,:]

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time,GHI
50,0,0,2.1,61.33,-10.8,0.0,9.5,0.258819,0.965926,22,1.0,0.0


In [31]:
#### testset과 dataset이 같은 모양인지 중간점검
dataset5.iloc[50:51,:]

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time,GHI
50,0,0,1.6,88.11,-11,0.0,9.0,0.258819,0.965926,23,1.0,0.0


In [32]:
testset6 = testset5[['DHI', 'DNI', 'WS', 'RH', 'T','SH', 'sin_time', 'cos_time', 'GHI', 'TARGET']] # 'DHI', 'DNI',
testset6.shape

(336, 10)

In [33]:
testset7 = pd.DataFrame(scaler.transform(testset6))
testset7

,0,1,2,3,4,5,6,7,8,9
0,-0.619312,-0.671447,0.241066,0.614298,-1.510851,-1.19583,1.300046e-17,1.414214,-0.74381,-0.690638
1,-0.619312,-0.671447,0.311150,0.688212,-1.540322,-1.19583,1.845919e-01,1.402115,-0.74381,-0.690638
2,-0.619312,-0.671447,0.311150,0.588450,-1.569792,-1.19583,3.660254e-01,1.366025,-0.74381,-0.690638
3,-0.619312,-0.671447,0.311150,0.687305,-1.609086,-1.19583,5.411961e-01,1.306563,-0.74381,-0.690638
4,-0.619312,-0.671447,0.311150,0.575300,-1.638557,-1.19583,7.071068e-01,1.224745,-0.74381,-0.690638
...,...,...,...,...,...,...,...,...,...,...
331,-0.619312,-0.671447,-0.529858,-0.595987,-1.422439,-1.19583,-8.609187e-01,1.121971,-0.74381,-0.690638
332,-0.619312,-0.671447,-0.670026,-0.733385,-1.392969,-1.19583,-7.071068e-01,1.224745,-0.74381,-0.690638
333,-0.619312,-0.671447,-0.670026,-0.733385,-1.392969,-1.19583,-5.411961e-01,1.306563,-0.74381,-0.690638
334,-0.619312,-0.671447,-0.670026,-0.797323,-1.392969,-1.19583,-3.660254e-01,1.366025,-0.74381,-0.690638


In [34]:
# testset7.columns = testset6.columns

testset8 = DataFrame(pca.transform(testset7))
testset8

,0,1,2,3,4,5,6
0,-2.365408,1.265302,-0.183554,0.193510,-0.080304,0.041526,-0.842242
1,-2.385631,1.337810,0.009197,0.222708,-0.063349,0.064250,-0.815847
2,-2.344561,1.400397,0.151608,0.192906,-0.025134,-0.051383,-0.815033
3,-2.368112,1.459922,0.325316,0.168618,0.037776,-0.004355,-0.748853
4,-2.305058,1.527248,0.447422,0.143384,0.086137,-0.123018,-0.715887
...,...,...,...,...,...,...,...
331,-1.904889,0.912187,-1.354055,-0.306877,0.067200,-0.735551,-0.928157
332,-1.906292,0.874076,-1.254126,-0.453420,0.102888,-0.905332,-1.029109
333,-1.937590,0.894355,-1.095938,-0.484054,0.105754,-0.953307,-1.086932
334,-1.937474,0.923410,-0.943711,-0.516199,0.111473,-1.052934,-1.143078


In [35]:
testset9 = test_to_supervised(testset8, n_days)
testset9

,0(t-144),1(t-144),2(t-144),3(t-144),4(t-144),5(t-144),6(t-144),0(t-143),1(t-143),2(t-143),3(t-143),4(t-143),5(t-143),6(t-143),0(t-142),1(t-142),2(t-142),3(t-142),4(t-142),5(t-142),6(t-142),0(t-141),1(t-141),2(t-141),3(t-141),4(t-141),5(t-141),6(t-141),0(t-140),1(t-140),2(t-140),3(t-140),4(t-140),5(t-140),6(t-140),0(t-139),1(t-139),2(t-139),3(t-139),4(t-139),...,2(t-6),3(t-6),4(t-6),5(t-6),6(t-6),0(t-5),1(t-5),2(t-5),3(t-5),4(t-5),5(t-5),6(t-5),0(t-4),1(t-4),2(t-4),3(t-4),4(t-4),5(t-4),6(t-4),0(t-3),1(t-3),2(t-3),3(t-3),4(t-3),5(t-3),6(t-3),0(t-2),1(t-2),2(t-2),3(t-2),4(t-2),5(t-2),6(t-2),0(t-1),1(t-1),2(t-1),3(t-1),4(t-1),5(t-1),6(t-1)
144,-2.365408,1.265302,-0.183554,0.193510,-0.080304,0.041526,-0.842242,-2.385631,1.337810,0.009197,0.222708,-0.063349,0.064250,-0.815847,-2.344561,1.400397,0.151608,0.192906,-0.025134,-0.051383,-0.815033,-2.368112,1.459922,0.325316,0.168618,0.037776,-0.004355,-0.748853,-2.305058,1.527248,0.447422,0.143384,0.086137,-0.123018,-0.715887,-2.322417,1.593293,0.602067,0.124805,0.162254,...,-1.185405,0.994681,-0.549959,-0.409883,-0.741508,-1.754370,1.375643,-1.040353,1.023828,-0.596230,-0.466193,-0.832890,-1.834312,1.384138,-0.869078,0.996137,-0.592050,-0.422298,-0.878764,-1.874453,1.382233,-0.722696,0.907894,-0.560168,-0.475795,-0.937544,-1.921824,1.398617,-0.542341,0.878600,-0.544411,-0.466524,-0.954815,-1.925927,1.425055,-0.377113,0.847739,-0.528713,-0.533493,-0.977882
145,-2.385631,1.337810,0.009197,0.222708,-0.063349,0.064250,-0.815847,-2.344561,1.400397,0.151608,0.192906,-0.025134,-0.051383,-0.815033,-2.368112,1.459922,0.325316,0.168618,0.037776,-0.004355,-0.748853,-2.305058,1.527248,0.447422,0.143384,0.086137,-0.123018,-0.715887,-2.322417,1.593293,0.602067,0.124805,0.162254,-0.048287,-0.614205,-2.248424,1.666446,0.697062,0.105690,0.220549,...,-1.040353,1.023828,-0.596230,-0.466193,-0.832890,-1.834312,1.384138,-0.869078,0.996137,-0.592050,-0.422298,-0.878764,-1.874453,1.382233,-0.722696,0.907894,-0.560168,-0.475795,-0.937544,-1.921824,1.398617,-0.542341,0.878600,-0.544411,-0.466524,-0.954815,-1.925927,1.425055,-0.377113,0.847739,-0.528713,-0.533493,-0.977882,-1.953549,1.450980,-0.192802,0.819260,-0.497498,-0.527230,-0.958705
146,-2.344561,1.400397,0.151608,0.192906,-0.025134,-0.051383,-0.815033,-2.368112,1.459922,0.325316,0.168618,0.037776,-0.004355,-0.748853,-2.305058,1.527248,0.447422,0.143384,0.086137,-0.123018,-0.715887,-2.322417,1.593293,0.602067,0.124805,0.162254,-0.048287,-0.614205,-2.248424,1.666446,0.697062,0.105690,0.220549,-0.151248,-0.552670,-2.241820,1.727783,0.815981,0.093948,0.298708,...,-0.869078,0.996137,-0.592050,-0.422298,-0.878764,-1.874453,1.382233,-0.722696,0.907894,-0.560168,-0.475795,-0.937544,-1.921824,1.398617,-0.542341,0.878600,-0.544411,-0.466524,-0.954815,-1.925927,1.425055,-0.377113,0.847739,-0.528713,-0.533493,-0.977882,-1.953549,1.450980,-0.192802,0.819260,-0.497498,-0.527230,-0.958705,-1.948859,1.463291,-0.039871,0.732771,-0.437540,-0.595674,-0.944148
147,-2.368112,1.459922,0.325316,0.168618,0.037776,-0.004355,-0.748853,-2.305058,1.527248,0.447422,0.143384,0.086137,-0.123018,-0.715887,-2.322417,1.593293,0.602067,0.124805,0.162254,-0.048287,-0.614205,-2.248424,1.666446,0.697062,0.105690,0.220549,-0.151248,-0.552670,-2.241820,1.727783,0.815981,0.093948,0.298708,-0.084744,-0.425761,-2.138803,1.801205,0.865931,0.081383,0.356581,...,-0.722696,0.907894,-0.560168,-0.475795,-0.937544,-1.921824,1.398617,-0.542341,0.878600,-0.544411,-0.466524,-0.954815,-1.925927,1.425055,-0.377113,0.847739,-0.528713,-0.533493,-0.977882,-1.953549,1.450980,-0.192802,0.819260,-0.497498,-0.527230,-0.958705,-1.948859,1.463291,-0.039871,0.732771,-0.437540,-0.595674,-0.944148,-1.978520,1.472205,0.132363,0.650225,-0.360543,-0.565239,-0.879890
148,-2.305058,1.527248,0.447422,0.143384,0.086137,-0.123018,-0.715887,-2.322417,1.593293,0.602067,0.124805,0.162254,-0.048287,-0.614205,-2.248424,1.666446,0.697062,0.105690,0.220549,-0.151248,-0.552670,-2.241820,1.727783,0.815981,0.093948,0.298708,-0.084744,-0

In [36]:
testset10 = testset9.iloc[-48:,:]

In [37]:
df_test = []

# 예시 코드
# 아직 유진이 어떻게 변수 추가할 지 몰라서 to_supervised는 쓰지 않았음.

for i in range(81):
    file_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/' + str(1) + '.csv'
    new_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/adj_test/' + str(1) + '.csv'
    file_name = str(1) + '.csv'

    # adj_test에서 가져와야 하므로 new_path에서 불러온다
    temp = pd.read_csv(new_path)

    temp.drop('Unnamed: 0', axis = 1, inplace = True)
    testset = temp.copy()

    # 24로 저장한 24절기 season으로 이름 바꾸기
    testset.rename(columns = {'24' : 'season'}, inplace = True)

    # 이 다음은 trainset과 동일한 전처리, season은 이미 구해져 있으므로 구하지 않음
    testset2 = drop_clms(testset)
    testset2.drop(['Hour', 'Minute','Date'], axis=1,inplace=True)
    testset3 = cos_time(testset2)
    testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]

    testset5 = testset4.copy()
    testset5['GHI'] = testset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)
    testset6 = testset5[['DHI', 'DNI', 'WS', 'RH', 'T','SH', 'sin_time', 'cos_time', 'GHI', 'TARGET']] # 'DHI', 'DNI',
    testset7 = pd.DataFrame(scaler.transform(testset6))
    testset8 = DataFrame(pca.transform(testset7))

    testset9 = test_to_supervised(testset8, n_days)
    testset10 = testset9.iloc[-48:,:]
    df_test.append(testset10)

X_test = pd.concat(df_test)

# X_test = X_test.iloc[:, :n_obs]
X_test 

,0(t-144),1(t-144),2(t-144),3(t-144),4(t-144),5(t-144),6(t-144),0(t-143),1(t-143),2(t-143),3(t-143),4(t-143),5(t-143),6(t-143),0(t-142),1(t-142),2(t-142),3(t-142),4(t-142),5(t-142),6(t-142),0(t-141),1(t-141),2(t-141),3(t-141),4(t-141),5(t-141),6(t-141),0(t-140),1(t-140),2(t-140),3(t-140),4(t-140),5(t-140),6(t-140),0(t-139),1(t-139),2(t-139),3(t-139),4(t-139),...,2(t-6),3(t-6),4(t-6),5(t-6),6(t-6),0(t-5),1(t-5),2(t-5),3(t-5),4(t-5),5(t-5),6(t-5),0(t-4),1(t-4),2(t-4),3(t-4),4(t-4),5(t-4),6(t-4),0(t-3),1(t-3),2(t-3),3(t-3),4(t-3),5(t-3),6(t-3),0(t-2),1(t-2),2(t-2),3(t-2),4(t-2),5(t-2),6(t-2),0(t-1),1(t-1),2(t-1),3(t-1),4(t-1),5(t-1),6(t-1)
288,-1.953549,1.450980,-0.192802,0.819260,-0.497498,-0.527230,-0.958705,-1.948859,1.463291,-0.039871,0.732771,-0.437540,-0.595674,-0.944148,-1.978520,1.472205,0.132363,0.650225,-0.360543,-0.565239,-0.879890,-1.944593,1.486168,0.268612,0.567501,-0.291736,-0.649291,-0.831277,-1.971125,1.476134,0.414277,0.432425,-0.173201,-0.605203,-0.728702,-1.940979,1.474263,0.520283,0.297858,-0.061902,...,-1.364739,-0.226242,0.226147,0.007185,-0.858807,-2.309249,1.216726,-1.227798,-0.254621,0.212496,-0.036049,-0.949123,-2.329285,1.228647,-1.087839,-0.284944,0.201230,-0.121417,-1.032516,-2.360583,1.248927,-0.929651,-0.315577,0.204096,-0.169391,-1.090339,-2.358699,1.272694,-0.776560,-0.347386,0.208165,-0.269352,-1.142648,-2.361964,1.272845,-0.626654,-0.435924,0.249352,-0.359726,-1.168463
289,-1.948859,1.463291,-0.039871,0.732771,-0.437540,-0.595674,-0.944148,-1.978520,1.472205,0.132363,0.650225,-0.360543,-0.565239,-0.879890,-1.944593,1.486168,0.268612,0.567501,-0.291736,-0.649291,-0.831277,-1.971125,1.476134,0.414277,0.432425,-0.173201,-0.605203,-0.728702,-1.940979,1.474263,0.520283,0.297858,-0.061902,-0.664569,-0.644739,-1.955926,1.470154,0.638480,0.168868,0.066413,...,-1.227798,-0.254621,0.212496,-0.036049,-0.949123,-2.329285,1.228647,-1.087839,-0.284944,0.201230,-0.121417,-1.032516,-2.360583,1.248927,-0.929651,-0.315577,0.204096,-0.169391,-1.090339,-2.358699,1.272694,-0.776560,-0.347386,0.208165,-0.269352,-1.142648,-2.361964,1.272845,-0.626654,-0.435924,0.249352,-0.359726,-1.168463,-2.316930,1.298554,-0.478873,-0.467258,0.260867,-0.507412,-1.190779
290,-1.978520,1.472205,0.132363,0.650225,-0.360543,-0.565239,-0.879890,-1.944593,1.486168,0.268612,0.567501,-0.291736,-0.649291,-0.831277,-1.971125,1.476134,0.414277,0.432425,-0.173201,-0.605203,-0.728702,-1.940979,1.474263,0.520283,0.297858,-0.061902,-0.664569,-0.644739,-1.955926,1.470154,0.638480,0.168868,0.066413,-0.606211,-0.514054,-1.891874,1.512337,0.734814,0.156326,0.124789,...,-1.087839,-0.284944,0.201230,-0.121417,-1.032516,-2.360583,1.248927,-0.929651,-0.315577,0.204096,-0.169391,-1.090339,-2.358699,1.272694,-0.776560,-0.347386,0.208165,-0.269352,-1.142648,-2.361964,1.272845,-0.626654,-0.435924,0.249352,-0.359726,-1.168463,-2.316930,1.298554,-0.478873,-0.467258,0.260867,-0.507412,-1.190779,-2.292910,1.329069,-0.318099,-0.496247,0.288090,-0.590007,-1.177632
291,-1.944593,1.486168,0.268612,0.567501,-0.291736,-0.649291,-0.831277,-1.971125,1.476134,0.414277,0.432425,-0.173201,-0.605203,-0.728702,-1.940979,1.474263,0.520283,0.297858,-0.061902,-0.664569,-0.644739,-1.955926,1.470154,0.638480,0.168868,0.066413,-0.606211,-0.514054,-1.891874,1.512337,0.734814,0.156326,0.124789,-0.649175,-0.402539,-1.864917,1.531287,0.817537,0.091289,0.222709,...,-0.929651,-0.315577,0.204096,-0.169391,-1.090339,-2.358699,1.272694,-0.776560,-0.347386,0.208165,-0.269352,-1.142648,-2.361964,1.272845,-0.626654,-0.435924,0.249352,-0.359726,-1.168463,-2.316930,1.298554,-0.478873,-0.467258,0.260867,-0.507412,-1.190779,-2.292910,1.329069,-0.318099,-0.496247,0.288090,-0.590007,-1.177632,-2.237187,1.366087,-0.174583,-0.524664,0.316678,-0.720202,-1.161769
292,-1.971125,1.476134,0.414277,0.432425,-0.173201,-0.605203,-0.728702,-1.940979,1.474263,0.520283,0.297858,-0.061902,-0.664569,-0.644739,-1.955926,1.470154,0.638480,0.168868,0.066413,-0.606211,-0.514054,-1.891874,1.512337,0.734814,0.156326,0.

In [38]:
X_train_1, X_valid_1, Y_train_1,  Y_valid_1 = train_test_split(X, y_1, test_size=0.30, random_state=42)
X_train_2, X_valid_2, Y_train_2,  Y_valid_2 = train_test_split(X, y_2, test_size=0.30, random_state=42)

quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [43]:
# LGBM 모델을 구축하고 예측까지 하는 함수를 만든다. 
def LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test):
    
    # (a) 모델링
    model = LGBMRegressor(objective='quantile', alpha=q,
                         n_estimators=10000, learning_rate=0.1, save_binary = True, subsample=0.7, device = 'gpu')          
    
        # bagging_fraction : 배깅을 하기위해서 데이터를 랜덤 샘플링하여 학습에 사용한다. 비율은 0 < fraction <= 1 이며 0이 되지 않게 해야한다.
        # learning_rate : 일반적으로 0.01 ~ 0.1 정도로 맞추고 다른 파라미터를 튜닝한다. 나중에 성능을 더 높일 때 learning rate를 더 줄인다.
        # n_estimators : 기본값은 100, 1000 정도로 해주는 게 좋고 많을 수록 과적합이 발생한 가능성이 높음.
        # save_binary = True 넣어주면 더 빨라진다고 함.
        # https://greatjoy.tistory.com/72
        
    model.fit(X_train, Y_train, eval_metric = ['quantile'], eval_set=[(X_valid, Y_valid)], early_stopping_rounds=500, verbose=500)


        # verbose : eval metric이 이 숫자만큼의 round가 지난 다음 자동으로 출력된다.
        # early_stopping_rounds : 이 숫자가 가기 전까지 validation score가 증가하지 않으면 round를 멈춘다. 

    # (b) 예측
    pred = pd.Series(model.predict(X_test).round(2))
    return pred, model

In [44]:
# Target 예측

def train_data(X_train, Y_train, X_valid, Y_valid, X_test):

    LGBM_models=[]
    LGBM_actual_pred = pd.DataFrame()

    for q in quantiles:
        print(q)
        pred, model = LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test)
        LGBM_models.append(model)
        LGBM_actual_pred = pd.concat([LGBM_actual_pred,pred],axis=1)

    LGBM_actual_pred.columns=quantiles
    
    return LGBM_models, LGBM_actual_pred

In [45]:
tick = time.time()

# Target1

models_1, results_1 = train_data(X_train_1, Y_train_1, X_valid_1, Y_valid_1, X_test)
results_1.sort_index()[:48]

0.1
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 1.37752
[1000]	valid_0's quantile: 1.36701
[1500]	valid_0's quantile: 1.34394
[2000]	valid_0's quantile: 1.33782
[2500]	valid_0's quantile: 1.33134
[3000]	valid_0's quantile: 1.32875
[3500]	valid_0's quantile: 1.32816
[4000]	valid_0's quantile: 1.32535
[4500]	valid_0's quantile: 1.32326
[5000]	valid_0's quantile: 1.32154
[5500]	valid_0's quantile: 1.32138
Early stopping, best iteration is:
[5051]	valid_0's quantile: 1.32095
0.2
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 2.0821
[1000]	valid_0's quantile: 2.02131
[1500]	valid_0's quantile: 1.98897
[2000]	valid_0's quantile: 1.98026
[2500]	valid_0's quantile: 1.95286
[3000]	valid_0's quantile: 1.93392
[3500]	valid_0's quantile: 1.93078
[4000]	valid_0's quantile: 1.92111
[4500]	valid_0's quantile: 1.91703
[5000]	valid_0's quantile: 1.91449
[5500]	valid_0's quantile: 1.91132
[6000]	valid_0's quantile: 1.90

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [46]:
# Target2

models_2, results_2 = train_data(X_train_2, Y_train_2, X_valid_2, Y_valid_2, X_test)
tock = time.time()
results_2.sort_index()

0.1
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 1.41702
[1000]	valid_0's quantile: 1.40681
[1500]	valid_0's quantile: 1.40083
[2000]	valid_0's quantile: 1.39581
[2500]	valid_0's quantile: 1.3926
Early stopping, best iteration is:
[2305]	valid_0's quantile: 1.39247
0.2
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 2.1437
[1000]	valid_0's quantile: 2.10701
[1500]	valid_0's quantile: 2.07262
[2000]	valid_0's quantile: 2.06433
[2500]	valid_0's quantile: 2.06068
[3000]	valid_0's quantile: 2.04392
[3500]	valid_0's quantile: 2.04161
[4000]	valid_0's quantile: 2.04071
[4500]	valid_0's quantile: 2.04027
Early stopping, best iteration is:
[4410]	valid_0's quantile: 2.04026
0.3
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 2.48158
[1000]	valid_0's quantile: 2.37912
[1500]	valid_0's quantile: 2.37523
[2000]	valid_0's quantile: 2.37188
[2500]	valid_0's quantile: 2.357
[300

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
3883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3885,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
print('소요시간 :', tock-tick)

print(results_1.shape, results_2.shape)

소요시간 : 6790.894842624664
(3888, 9) (3888, 9)


In [48]:
# 출력

submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = results_1.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = results_2.sort_index().values
submission2 = submission.set_index('id')
submission3 = not_minus(submission2)

In [49]:
submission3.to_csv('submission14.csv')
!cp submission14.csv "drive/My Drive/"